# Features selection with STEPWISE

In [2]:
import sys

my_path = r'/home/ilaria/Scrivania/Machine_Learning/Project_1/Project1_ML'
sys.path.insert(0,my_path + r'/code/COMMON')

import numpy as np 
import matplotlib.pyplot as plt
from proj1_helpers import load_csv_data, predict_labels 
from implementations import least_squares
from outliers import handle_outliers
from labels import idx_2labels
from standard import standardize

In [3]:
yb, input_data, ids = load_csv_data(my_path + r'/data/train.csv', sub_sample=False)

In [4]:
input_data.shape

(250000, 30)

In [5]:
input_data, Y = handle_outliers(input_data,yb,-999,0) # substiution with zeros
ind_back, ind_sig = idx_2labels(Y, [-1,1])

input_data, mean_X, std_X = standardize(input_data)    

-999 are replaced by 0


In [6]:
# Subdived the X features space in single features
all_features = np.genfromtxt(my_path + r'/data/train.csv', delimiter=",", dtype=str, max_rows = 1)[2:]

features = []
for i in range(len(all_features)):
    features.append((i,all_features[i]))

## R^2  as stopping criteria : R2 of Tjur or R2 of McFadden

The only way to use the stepwise is using R2 of Tjur or Cox and Snell or McFadden because of the binary values of the indipendent variable

### Least square model

#### Use of the error before binarization (R2 with loss)

In [7]:
# Start of STEP-WISE algorithm 
all_candidates = input_data

n = all_candidates.shape[0] #needed for the R^2 adjusted
num = all_candidates.shape[1]
H = np.ones((n,1)) #offset

#Initialization only with offsets (lack of info)
X = H
K = 0 #needed for the R^2 adjusted

w0, loss = least_squares(Y,X)  # The loss cannot be used as a measure for the feature selection because it's a 
y = predict_labels(w0, X)

sse = loss
sst = np.sum((Y - Y.mean())**2)  #lack of information
R2 = np.abs((sst-sse)/sst)
R2adj_0 = R2 - (K/(n-K-1)*(1-R2))

#fix the R2adj_max

R2adj_max = R2adj_0
ind_max = 0  # this index will show us which is the best feature chosen
del(X)
idx_features = []
best_R2adj = []

for j in range(num):
    R2_adj = []
    for i in range(all_candidates.shape[1]):
        
        X = np.concatenate((H,all_candidates[:,i].reshape(n,1)), axis=1)
        ws , loss = least_squares(Y,X)
        k = len(ws) -1 # k is the number of regressor I use -> -1 because I don't consider the offset
        
        #y = predict_labels(ws,X)   #***** NO USE OF PREDICTION
        SSE = np.sum(loss**2)
        SST = np.sum((Y- Y.mean())**2)
        R2 = np.abs((SST-SSE)/SST)
        R2_adj.append(R2 - (k/(n-k-1)*(1-R2)))
        
    R2adj_chosen = np.max(R2_adj)
    best_R2adj.append(R2adj_chosen)
    idx_chosen = np.argmax(R2_adj)
    
    if R2adj_chosen > R2adj_max:
        R2adj_max = R2adj_chosen
        ind_max = idx_chosen
        
        H = np.concatenate((H, all_candidates[:,ind_max].reshape(n,1)), axis = 1)
        
        all_candidates = np.delete(all_candidates,ind_max,1)
        print('-------------------------------------------------')
        print('Feature chosen: ', features[ind_max][1])
        idx_features.append(features[ind_max][0])
        del(features[ind_max])
        
        del(X)
        
    else:
        break
        

-------------------------------------------------
Feature chosen:  DER_mass_transverse_met_lep
-------------------------------------------------
Feature chosen:  PRI_tau_pt
-------------------------------------------------
Feature chosen:  DER_prodeta_jet_jet
-------------------------------------------------
Feature chosen:  DER_met_phi_centrality
-------------------------------------------------
Feature chosen:  DER_deltar_tau_lep
-------------------------------------------------
Feature chosen:  DER_mass_vis
-------------------------------------------------
Feature chosen:  PRI_lep_pt
-------------------------------------------------
Feature chosen:  DER_pt_ratio_lep_tau
-------------------------------------------------
Feature chosen:  DER_lep_eta_centrality
-------------------------------------------------
Feature chosen:  PRI_met_sumet
-------------------------------------------------
Feature chosen:  PRI_met
-------------------------------------------------
Feature chosen:  DER_m

In [8]:
R2_adj

[0.99999948969730612,
 0.99999948969692165,
 0.99999948969636787,
 0.99999948969669772,
 0.99999948969569463,
 0.99999948969568886,
 0.99999948969751906]

In [9]:
print("Number of features chosen:", len(idx_features))
print(idx_features)

Number of features chosen: 23
[1, 13, 6, 11, 7, 2, 16, 10, 12, 21, 19, 5, 9, 23, 0, 26, 3, 22, 4, 18, 28, 29, 8]


#### Use of the probability of the 2 events (R2 Tjur)

In [10]:
# Realloc FEATURES
features = []
for i in range(len(all_features)):
    features.append((i,all_features[i]))


In [11]:
# Start of STEP-WISE algorithm 
all_candidates = input_data

n = all_candidates.shape[0] #needed for the R^2 adjusted
num = all_candidates.shape[1]
H = np.ones((n,1)) #offset

#Initialization only with offsets (lack of info)
X = H
K = 0 #needed for the R^2 adjusted

w0, loss = least_squares(Y,X)  # The loss cannot be used as a measure for the feature selection because it's a 
#y = predict_labels(w0, X)
ind_back, ind_sig = idx_2labels(y, [0,1])

y_ = X.dot(w0)
R2 = 0
R2adj_0 = R2 - (K/(n-K-1)*(1-R2))

#fix the R2adj_max
R2adj_max = R2adj_0
ind_max = 0  # this index will show us which is the best feature chosen
del(X)
idx_features = []
best_R2adj = []

for j in range(num):
    R2_adj = []
    for i in range(all_candidates.shape[1]):
        
        X = np.concatenate((H,all_candidates[:,i].reshape(n,1)), axis=1)
        ws , loss = least_squares(Y,X)
        k = len(ws) -1 # k is the number of regressor I use -> -1 because I don't consider the offset
        
        y = predict_labels(ws,X)          
        ind_back, ind_sig = idx_2labels(y, [-1,1])
        
        if len(ind_sig) == 0 or len(ind_back) ==0:
            print('No signal detected')
            R2_adj.append(0)
            
        else: 
            
            y_ = X.dot(ws)

            R2 = np.abs((np.mean(y_[ind_sig]) - np.mean(y_[ind_back])))
            R2_adj.append(R2 - (k/(n-k-1)*(1-R2)))
            
        
    R2adj_chosen = np.max(R2_adj)
    best_R2adj.append(R2adj_chosen)
    idx_chosen = np.argmax(R2_adj)

    if R2adj_chosen > R2adj_max:
        R2adj_max = R2adj_chosen
        ind_max = idx_chosen

        #idx_features.append(np.where(all_candidates[:,ind_max] == input_data))
        H = np.concatenate((H, all_candidates[:,ind_max].reshape(n,1)), axis = 1)

        all_candidates = np.delete(all_candidates,ind_max,1)
        print('-------------------------------------------------')
        print('Feature chosen: ', features[ind_max][1])
        idx_features.append(features[ind_max][0])
        del(features[ind_max])

        del(X)

    else:
        break

No signal detected
No signal detected
No signal detected
No signal detected
No signal detected
No signal detected
No signal detected
No signal detected
No signal detected
No signal detected
No signal detected
No signal detected
No signal detected
No signal detected
No signal detected
-------------------------------------------------
Feature chosen:  DER_mass_jet_jet
-------------------------------------------------
Feature chosen:  PRI_jet_subleading_pt
-------------------------------------------------
Feature chosen:  PRI_tau_pt
-------------------------------------------------
Feature chosen:  DER_mass_vis
-------------------------------------------------
Feature chosen:  DER_pt_h
-------------------------------------------------
Feature chosen:  DER_sum_pt
-------------------------------------------------
Feature chosen:  DER_pt_tot
-------------------------------------------------
Feature chosen:  PRI_met
-------------------------------------------------
Feature chosen:  PRI_met_su

In [12]:
R2_adj

[0.79932954505553233,
 0.73814002387044686,
 0.81078963226025014,
 0.8167394283129078,
 0.7834495977228878,
 0.75618167933603697,
 0.81110238707071369,
 0.82140638806704502,
 0.82068400833700905,
 0.81451288049716175,
 0.82087527683274886,
 0.82083313346174369,
 0.81195590681577445,
 0.80959831329100285,
 0.82134458520180198,
 0.82151163207493982,
 0.81451290130685072]

In [13]:
print("Number of features chosen:", len(idx_features))
print(idx_features)

Number of features chosen: 13
[5, 26, 13, 2, 3, 9, 8, 19, 21, 17, 6, 28, 27]


#### Use the likelihood (McFadden)

In [14]:
# Realloc FEATURES
features = []
for i in range(len(all_features)):
    features.append((i,all_features[i]))
   

In [15]:
# Start of STEP-WISE algorithm 
all_candidates = input_data

n = all_candidates.shape[0] #needed for the R^2 adjusted
num = all_candidates.shape[1]
H = np.ones((n,1)) #offset

#Initialization only with offsets (lack of info)
X = H
K = 0 #needed for the R^2 adjusted

w0, loss = least_squares(Y,X)  # The loss cannot be used as a measure for the feature selection because it's a 
y = predict_labels(w0, X)
loglike0 = np.sum(np.log(1+np.exp(X.dot(w0))) - y*(X.dot(w0)))

R2 = 0        # For the definition of Cox and snell 1-1 = 0
R2adj_0 = 0

#fix the R2adj_max

R2adj_max = R2adj_0
ind_max = 0  # this index will show us which is the best feature chosen
del(X)
idx_features = []
best_R2adj = []

for j in range(num):
    R2_adj = []
    for i in range(all_candidates.shape[1]):
        
        X = np.concatenate((H,all_candidates[:,i].reshape(n,1)), axis=1)
        ws , loss = least_squares(Y,X)
        k = len(ws) -1 # k is the number of regressor I use -> -1 because I don't consider the offset
        
        y = predict_labels(ws,X)   
        
        loglike = np.sum(np.log(1+np.exp(X.dot(ws))) - y*(X.dot(ws)))
        
        R2 = 1-(loglike/loglike0)
        R2_adj.append(R2 - (k/(n-k-1)*(1-R2)))
        
    R2adj_chosen = np.max(R2_adj)
    best_R2adj.append(R2adj_chosen)
    idx_chosen = np.argmax(R2_adj)
    
    if R2adj_chosen > R2adj_max:
        R2adj_max = R2adj_chosen
        ind_max = idx_chosen
        
        H = np.concatenate((H, all_candidates[:,ind_max].reshape(n,1)), axis = 1)
        
        all_candidates = np.delete(all_candidates,ind_max,1)
        print('-------------------------------------------------')
        print('Feature chosen: ', features[ind_max][1])
        idx_features.append(features[ind_max][0])
        del(features[ind_max])
        
        del(X)
        
    else:
        break
        

-------------------------------------------------
Feature chosen:  DER_lep_eta_centrality
-------------------------------------------------
Feature chosen:  PRI_tau_pt
-------------------------------------------------
Feature chosen:  DER_mass_transverse_met_lep
-------------------------------------------------
Feature chosen:  DER_met_phi_centrality
-------------------------------------------------
Feature chosen:  PRI_jet_num
-------------------------------------------------
Feature chosen:  DER_deltaeta_jet_jet
-------------------------------------------------
Feature chosen:  PRI_met
-------------------------------------------------
Feature chosen:  PRI_jet_all_pt
-------------------------------------------------
Feature chosen:  PRI_lep_pt
-------------------------------------------------
Feature chosen:  DER_pt_h
-------------------------------------------------
Feature chosen:  PRI_jet_leading_pt
-------------------------------------------------
Feature chosen:  DER_mass_vis
---

In [16]:
R2_adj

[0.50113697844029692,
 0.50251753116391462,
 0.50368664230321181,
 0.4996807773808416,
 0.50371421645983028,
 0.50371273971310371,
 0.50371296716950276,
 0.5037141726137806,
 0.50371071437142145]

In [17]:
print("Number of features chosen:", len(idx_features))
print(idx_features)

Number of features chosen: 21
[12, 13, 1, 11, 22, 4, 19, 29, 16, 3, 23, 2, 7, 6, 21, 26, 18, 28, 9, 20, 14]


### Least square model using cross validation

#### R2 with likelihood (McFadden)

In [18]:
from split_data import split_data
from build_poly import build_poly

ratio = 0.7
seed = 1


In [17]:
# Realloc FEATURES
features = []
for i in range(len(all_features)):
    features.append((i,all_features[i]))

In [ ]:
# Start of STEP-WISE algorithm 
all_candidates = input_data

n = all_candidates.shape[0] #needed for the R^2 adjusted
num = all_candidates.shape[1]
H = np.ones((n,1)) #offset

#Initialization only with offsets (lack of info)
X = H
K = 0 #needed for the R^2 adjusted

w0, loss = least_squares(Y,X)  # The loss cannot be used as a measure for the feature selection because it's a 
y = predict_labels(w0, X)
loglike0 = np.sum(np.log(1+np.exp(X.dot(w0))) - y*(X.dot(w0)))

R2 = 0        # For the definition of Cox and snell 1-1 = 0
R2adj_0 = 0

#fix the R2adj_max

R2adj_max = R2adj_0
ind_max = 0  # this index will show us which is the best feature chosen
del(X)
idx_features = []
best_R2adj = []

for j in range(num):
    R2_adj = []
    for i in range(all_candidates.shape[1]):
        
        X = np.concatenate((H,all_candidates[:,i].reshape(n,1)), axis=1)
        ws , loss = least_squares(Y,X)
        k = len(ws) -1 # k is the number of regressor I use -> -1 because I don't consider the offset
        
        y = predict_labels(ws,X)   
        
        loglike = np.sum(np.log(1+np.exp(X.dot(ws))) - y*(X.dot(ws)))
        
        R2 = 1-(loglike/loglike0)
        R2_adj.append(R2 - (k/(n-k-1)*(1-R2)))
        
    R2adj_chosen = np.max(R2_adj)
    best_R2adj.append(R2adj_chosen)
    idx_chosen = np.argmax(R2_adj)
    
    if R2adj_chosen > R2adj_max:
        R2adj_max = R2adj_chosen
        ind_max = idx_chosen
        
        H = np.concatenate((H, all_candidates[:,ind_max].reshape(n,1)), axis = 1)
        
        all_candidates = np.delete(all_candidates,ind_max,1)
        print('-------------------------------------------------')
        print('Feature chosen: ', features[ind_max][1])
        idx_features.append(features[ind_max][0])
        del(features[ind_max])
        
        del(X)
        
    else:
        break